In [ ]:
from nasbench import api
%load_ext autoreload
%autoreload 2

In [ ]:
nasbench = api.NASBench('nasbench_only108.tfrecord')

In [ ]:
# Standard imports
import copy
import numpy as np
import matplotlib.pyplot as plt
import random

# Useful constants
INPUT = 'input'
OUTPUT = 'output'
CONV3X3 = 'conv3x3-bn-relu'
CONV1X1 = 'conv1x1-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'
NUM_VERTICES = 7
MAX_EDGES = 9
EDGE_SPOTS = NUM_VERTICES * (NUM_VERTICES - 1) / 2   # Upper triangular matrix
OP_SPOTS = NUM_VERTICES - 2   # Input/output vertices are fixed
ALLOWED_OPS = [CONV3X3, CONV1X1, MAXPOOL3X3]
ALLOWED_EDGES = [0, 1]   # Binary adjacency matrix

In [ ]:
# Query an Inception-like cell from the dataset.
cell = api.ModelSpec(
  matrix=[[0, 1, 1, 1, 0, 1, 0],    # input layer
          [0, 0, 0, 0, 0, 0, 1],    # 1x1 conv
          [0, 0, 0, 0, 0, 0, 1],    # 3x3 conv
          [0, 0, 0, 0, 1, 0, 0],    # 5x5 conv (replaced by two 3x3's)
          [0, 0, 0, 0, 0, 0, 1],    # 5x5 conv (replaced by two 3x3's)
          [0, 0, 0, 0, 0, 0, 1],    # 3x3 max-pool
          [0, 0, 0, 0, 0, 0, 0]],   # output layer
  # Operations at the vertices of the module, matches order of matrix.
  ops=[INPUT, CONV1X1, CONV3X3, CONV3X3, CONV3X3, MAXPOOL3X3, OUTPUT])

# Querying multiple times may yield different results. Each cell is evaluated 3
# times at each epoch budget and querying will sample one randomly.
data = nasbench.query(cell) 
print(type(data))
for k, v in data.items():
  print('%s: %s' % (k, str(v)))

# Create NAS problem

In [ ]:
import sys
sys.path.append('/home/zss/EMOC')

from emoc.problem import Problem
import random

class NASBench101(Problem):
    def __init__(self, vertice_num:int, objects:list, dataset_file:str):
        """NASBench101

        Args:
            vertice_num (int): the number of vertices
            objects (list): a list of objects to be optimized
            dataset_file (str): file path of dataset
        """
        if len(objects) < 2:
            raise Exception('objects must be more than 2!')
        for obj in objects:
            if obj not in ['trainable_parameters', 'training_time', 'train_accuracy', 'validation_accuracy', 'test_accuracy']:
                raise Exception('obj must be in [trainable_parameters, training_time, train_accuracy, validation_accuracy, test_accuracy')
        # self.nasbench = api.NASBench(dataset_file)
        self.nasbench = nasbench
        self.matrix_num_ = int(vertice_num * (vertice_num - 1) / 2)
        self.ops_num_ = vertice_num - 2
        self.dec_num_ = self.matrix_num_ + self.ops_num_
        self.obj_num_ = len(objects)
        super().__init__(self.dec_num_, self.obj_num_)
        
        self.objects = objects
        self.vertice_num_ = vertice_num
        self.problem_name_ = 'NASBench101'
        self.lower_bound_ = [0] * self.dec_num_
        
        
        upper_bound_ = [1] * self.dec_num_
        for i in range(self.matrix_num_, self.dec_num_):
            upper_bound_[i] = 2
        self.upper_bound_ = upper_bound_
        
        self.ops = ['conv3x3-bn-relu', 'conv1x1-bn-relu', 'maxpool3x3', 'input', 'output']
        self.encoding_ = self.GetType("INTEGER")
        
    
    def CalObj(self, ind):
        """CalObj

        Args:
            ind (emoc.Individual): individual to be evaluated
        """
        cell = self.Encoding(ind)
        data = nasbench.query(cell)
        objs = []
        for i in range(self.obj_num_):
            objs.append(data[self.objects[i]])
        ind.obj_ = objs
        
    def Encoding(self, ind):
        """Encoding
        Transform an individual to a model spec which can be queried by NASBench101

        Args:
            ind (emoc.Individual): individual to be encoded

        Returns:
            _type_: the model spec
        """
        matrix = np.zeros((self.vertice_num_, self.vertice_num_), dtype=np.int8)
        index = 0
        for i in range(self.vertice_num_):
            for j in range(i+1, self.vertice_num_):
                matrix[i][j] = ind.dec_[index]
                index += 1
        ops = ['input']
        for i in range(self.ops_num_):
            ops.append(self.ops[int(ind.dec_[index])])
        ops.append('output')
        return api.ModelSpec(matrix=matrix, ops=ops)
    
    def check(self, ind):
        """check
        check if an individual is valid

        Args:
            ind (emoc.Individual): individual to be checked

        Returns:
            bool: True if valid, False otherwise
        """
        cell = self.Encoding(ind)
        return self.nasbench.is_valid(cell)

## Sampling

In [ ]:
from emoc.operator import Operator

class sampling_NASBench101(Operator):
    def sample_ind(self, ind, problem):
        while True:
            dec = []
            for i in range(problem.dec_num_):
                dec.append(random.randint(problem.lower_bound_[i], problem.upper_bound_[i]))
            ind.dec_ = dec
            if problem.check(ind):
                break

# class sampling_NASBench101:
#     def __init__(self) -> None:
#         pass
    
#     def __call__(self, population, pop_num, problem):
#         for i in range(pop_num):
#             self.sample_ind(population[i], problem)
    
#     def sample_ind(self, ind, problem):
#         while True:
#             dec = []
#             for i in range(problem.dec_num_):
#                 dec.append(random.randint(problem.lower_bound_[i], problem.upper_bound_[i]))
#             ind.dec_ = dec
#             if problem.check(ind):
#                 break

# Mutation

In [ ]:
from emoc.mutation import Mutation

class mutation_NASBench101(Mutation):
    def mutation_ind(self, ind, problem):
        while True:
            for i in range(len(ind.dec_)):
                if random.random() < self.pro:
                    if i < problem.matrix_num_:
                        ind.dec_[i] = 1 - ind.dec_[i]
                    else:
                        ind.dec_[i] = random.randint(0, 2)
            if problem.check(ind):
                break

# class mutation_NASBench101:
#     def __init__(self, mu_pro=None):
#         self.pro = mu_pro
        
#     def __call__(self, population, pop_num, problem):
#         if self.pro == None:
#             self.pro = 1.0 / len(population[0].dec_)
#         for i in range(pop_num):
#             self.mutation_ind(population[i], problem)
    
#     def mutation_ind(self, ind, problem):
#         while True:
#             for i in range(len(ind.dec_)):
#                 if random.random() < self.pro:
#                     if i < problem.matrix_num_:
#                         ind.dec_[i] = 1 - ind.dec_[i]
#                     else:
#                         ind.dec_[i] = random.randint(0, 2)
#             if problem.check(ind):
#                 break

# Crossover

In [ ]:
from emoc.operator import Crossover:
class crossover_NASBench101(Crossover):
    def cross_ind(self, parent1, parent2, child1, child2, problem):
        if random.random() > self.pro:
            child1.dec_ = copy.deepcopy(parent1.dec_)
            child2.dec_ = copy.deepcopy(parent2.dec_)
            return
        while True:
            dec1 = []
            dec2 = []
            for i in range(len(parent1.dec_)):
                if random.random() < 0.5:
                    dec1.append(parent1.dec_[i])
                    dec2.append(parent2.dec_[i])
                    # child1.dec_[i] = parent1.dec_[i]
                    # child2.dec_[i] = parent2.dec_[i]
                else:
                    dec1.append(parent2.dec_[i])
                    dec2.append(parent1.dec_[i])
                child1.dec_ = dec1
                child2.dec_ = dec2
            if problem.check(child1) and problem.check(child2):
                break
            

# class crossover_NASBench101:
#     def __init__(self, cross_pro = 0.9):
#         self.pro = cross_pro
    
#     def __call__(self, parent_pop, offspring_pop, pop_num, problem, selection_operator, **kwargs):
#         index1 = list(range(pop_num))
#         random.shuffle(index1)
#         index2 = list(range(pop_num))
#         random.shuffle(index2)
#         for i in range(pop_num // 2):
#             parent1 = selection_operator(parent_pop[index1[2 * i]], parent_pop[index1[2 * i + 1]])
#             parent2 = selection_operator(parent_pop[index2[2 * i]], parent_pop[index2[2 * i + 1]])
#             self.cross_ind(parent1, parent2, offspring_pop[2 * i], offspring_pop[2 * i + 1], problem)
            
#     def cross_ind(self, parent1, parent2, child1, child2, problem):
#         if random.random() > self.pro:
#             child1.dec_ = copy.deepcopy(parent1.dec_)
#             child2.dec_ = copy.deepcopy(parent2.dec_)
#             return
#         while True:
#             dec1 = []
#             dec2 = []
#             for i in range(len(parent1.dec_)):
#                 if random.random() < 0.5:
#                     dec1.append(parent1.dec_[i])
#                     dec2.append(parent2.dec_[i])
#                     # child1.dec_[i] = parent1.dec_[i]
#                     # child2.dec_[i] = parent2.dec_[i]
#                 else:
#                     dec1.append(parent2.dec_[i])
#                     dec2.append(parent1.dec_[i])
#                 child1.dec_ = dec1
#                 child2.dec_ = dec2
#             if problem.check(child1) and problem.check(child2):
#                 break

In [ ]:
from emoc.algorithm import NSGA2Framework
from emoc.core import EMOC_Manager

myAlgorithm = NSGA2Framework(sampling=sampling_NASBench101(), mutation=mutation_NASBench101(), crossover=crossover_NASBench101(cross_pro = 0.8))
emoc = EMOC_Manager(population_num=100, max_evaluation=25000, run=5)
nas = NASBench101(vertice_num=7, objects=['test_accuracy', 'training_time'], dataset_file='nasbench_only108.tfrecord')
emoc.optimize(myAlgorithm, nas, metrics=[], output_interval= 50)

In [ ]:
emoc.global_[0].record_[5].pop_[2].dec_, emoc.global_[0].record_[5].pop_[2].obj_